In [1]:
import requests
import dataset

In [2]:
db = dataset.connect("sqlite:///article_metadata.sqlite")
tab = db['articles']

In [23]:
def extract_tags(row):
    tags = [w.lower().strip().replace("&","and").replace(" ","-").replace("/","-") for w in row['keywords'].split(",")]
    tags = [t for t in test_tags if t != '']
    
    countries = [w.lower().strip().replace("&","and").replace(" ","-").replace("/","-") for w in row['countries'].split(",")] 
    countries = [c for c in countries if c != '']
    
    year = [row['date_published'].split(" ")[-1]]
    
    return year + countries + tags

In [4]:
r

OrderedDict([('id', 1),
             ('keywords', 'Conflict, violence & peace, Health, Refugees/IDPs'),
             ('agency', 'The Humanitarian Health Ethics Network'),
             ('about',
              '{"Pages": "35pp", "Language": "English", "Keywords": "Conflict, violence & peace, Health, Refugees/IDPs", "Agency": "The Humanitarian Health Ethics Network", "Date published": "24 May 2017", "Resource type": "Research, reports and studies", "Countries": "Jordan, Syria", "Author(s)": "McDonald, M. "}'),
             ('isbn', ''),
             ('publisher', ''),
             ('date_published', '24 May 2017'),
             ('resource_type', 'Research, reports and studies'),
             ('language', 'English'),
             ('authors', 'McDonald, M. '),
             ('countries', 'Jordan, Syria'),
             ('description',
              'The Hashemite Kingdom of Jordan shares its northern border with the Syrian Arab Republic, and has been one of the main receiving countries of fle

In [24]:
api_key = 'REDACTED'

In [26]:
# Testing is category id 5
# Resources ==> Report is category id 15

payload = {"category":15,
           "title":"API Test with tags from system",
           "created_at":"2017-06-02",
           "raw":"**Bold** new topic first test post. [Test link](http://www.google.com)\n\nok\n\nanother line",
           "skip_validations":True,
           "tags[]":["syria","meta","auto-import"]}

response = requests.request("POST","https://aublcs.fxbprojects.org/posts",
                        headers={"Content-Type":"multipart/form-data"},
                        params={"api_key":api_key,"api_username":"system"},
                        data=payload
                        )
response.json()

{'actions_summary': [{'can_act': True, 'id': 3},
  {'can_act': True, 'id': 4},
  {'can_act': True, 'hidden': True, 'id': 5},
  {'can_act': True, 'id': 7},
  {'can_act': True, 'id': 8}],
 'admin': True,
 'avatar_template': '/user_avatar/aublcs.fxbprojects.org/system/{size}/1_1.png',
 'avg_time': None,
 'can_delete': False,
 'can_edit': True,
 'can_recover': None,
 'can_view_edit_history': True,
 'can_wiki': True,
 'cooked': '<p><strong>Bold</strong> new topic first test post. <a href="http://www.google.com">Test link</a></p>\n\n<p>ok</p>\n\n<p>another line</p>',
 'created_at': '2017-06-02T00:00:00.000Z',
 'deleted_at': None,
 'display_username': 'system',
 'draft_sequence': 0,
 'edit_reason': None,
 'hidden': False,
 'hidden_reason_id': None,
 'id': 233,
 'incoming_link_count': 0,
 'moderator': True,
 'name': 'system',
 'post_number': 1,
 'post_type': 1,
 'primary_group_flair_bg_color': None,
 'primary_group_flair_color': None,
 'primary_group_flair_url': None,
 'primary_group_name': No

In [44]:
fields_lookup = {'agency': 'Agency',
                 'authors': 'Author(s)',
                 'countries': 'Countries',
                 'date_published': 'Date published',
                 'keywords': 'Keywords',
                 'language': 'Language',
                 'pages': 'Pages',
                 'publisher': 'Publisher',
                 'resource_type': 'Resource type',
                 'isbn':'ISBN',
                 'description':'Description',
                 'link':'URL'}

def post_record(row):
    payload = {"category":15,
               "title":row['title'],
               "created_at":"2017-05-10",
               "skip_validations":True}
    
    tags = extract_tags(row)
    
    if len(tags) > 0:
        payload["tags[]"] = tags
    
    content = "**Title**\n" + row['title'] + "\n\n"
    
    for key in row.keys():
        if key == "about" or key == "title" or key == "id" or key == 'link':
            continue
        if row[key].strip() == '':
            continue
        clean_key = key
        
        try:
            clean_key = fields_lookup[key]
        except:
            pass
        
        content += "**" + clean_key + "**\n" + row[key].strip() + "\n\n"
        
    content += "**URL**\n`" + row['link'] + "`\n\n" + row['link']
        
    payload['raw'] = content
        
    response = requests.request("POST","https://aublcs.fxbprojects.org/posts",
                            headers={"Content-Type":"multipart/form-data"},
                            params={"api_key":api_key,"api_username":"system"},
                            data=payload
                            )
    if response.status_code != 200:
        print("ID",row['id'],"failed")
        return -1
    else:
        return response.json()['topic_id']

In [45]:
# Tracks insert status. If the topic_id is -1 then the insert failed.
post_tracker = {}

In [46]:
for r in tab.find():
    topic_id = post_record(r)
    post_tracker[r['id']] = topic_id

ID 159 failed
ID 317 failed
ID 344 failed
ID 518 failed
ID 550 failed
ID 559 failed
ID 817 failed
ID 856 failed
ID 1080 failed
ID 1090 failed
ID 1106 failed
ID 1154 failed
ID 1201 failed
ID 1383 failed


In [47]:
for k,v in post_tracker.items():
    if v == -1:
        print(k)

159
317
344
518
550
559
817
856
1080
1090
1106
1154
1201
1383


In [48]:
[k for k,v in post_tracker.items() if v == -1]

[159, 317, 344, 518, 550, 559, 817, 856, 1080, 1090, 1106, 1154, 1201, 1383]